In [1]:
##Importing packages we need##

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.animation import FuncAnimation
from matplotlib import cm
from matplotlib.cm import get_cmap
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
import metpy.calc as mpcalc
from metpy.units import units
from numpy import *
import xarray as xr
from netCDF4 import Dataset, num2date
import math
import pygrib
import cdsapi
import imageio
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import metpy as mp
import warnings
import glob
import dask
warnings.filterwarnings("ignore")

print("Done importing modules")

Done importing modules


In [7]:
##Get all the ERA5 files##

#Organize the files with glob#
path_pattern = 'CF_DATA/ERA5*.nc'
files = glob.glob(path_pattern, recursive=True)

#Sort them in alphabetical order#
files_sorted = sorted(files)
files_sorted[5]

#Create the directory for sliced files#
sliced_directory = 'Data_Remake_Sliced'
os.makedirs(sliced_directory, exist_ok=True)

In [12]:
##Function to check if a file is an edge case and determine the slicing##

def get_slice_indices(file_name):
    '''Extract the date part from the file name and slice based on edge case.'''
    
    try:
        date_str = os.path.basename(file_name).split('_')[1:]  #Example: '01_02_1952'
        date_str = "_".join(date_str).split('.')[0]   #Join parts and remove the file extension
        date = pd.to_datetime(date_str, format='%m_%d_%Y')  #format to datetime
        
        #Check for edge cases and return the corresponding slice indices#
        if date.month == 1 and date.day == 1:
            return slice(96, 168)
        
        elif date.month == 1 and date.day == 2:
            return slice(120, 192)
        
        elif date.day == 1:
            return slice(24, 96)
        
        elif date.day == 2:
            return slice(48, 120)
        
        else:
            return None
        
    except Exception as e:
        
        print(f"Error parsing date from file {file_name}: {e}")
        return None

#Loop through each file, check if it's an edge case, slice and save the subset#
for file in files_sorted:
    
    slice_indices = get_slice_indices(file)
    
    if slice_indices is not None:
        
        ds = xr.open_dataset(file).metpy.parse_cf()
        ds_sliced = ds.isel(time=slice_indices)
        
        #Exclude 'metpy_crs' variable before saving#
        ds_sliced = ds_sliced.drop_vars('metpy_crs', errors='ignore')
        
        #Define the new file name for the sliced dataset#
        new_file_name = os.path.join(sliced_directory, os.path.basename(file))
        
        #Save the sliced dataset#
        ds_sliced.to_netcdf(new_file_name)
        print(f"Saved sliced file: {new_file_name}")
        
    else:
        print(f"File {file} is not an edge case and does not need slicing.")

print("Processing complete.")

Saved sliced file: Data_Remake_Sliced/ERA5_01_01_2011.nc
Saved sliced file: Data_Remake_Sliced/ERA5_01_01_2017.nc
Saved sliced file: Data_Remake_Sliced/ERA5_01_02_1951.nc
Saved sliced file: Data_Remake_Sliced/ERA5_01_02_1952.nc
Saved sliced file: Data_Remake_Sliced/ERA5_01_02_2022.nc
File CF_DATA/ERA5_01_03_1959.nc is not an edge case and does not need slicing.
File CF_DATA/ERA5_01_03_1991.nc is not an edge case and does not need slicing.
File CF_DATA/ERA5_01_04_1950.nc is not an edge case and does not need slicing.
File CF_DATA/ERA5_01_04_1959.nc is not an edge case and does not need slicing.
File CF_DATA/ERA5_01_04_1972.nc is not an edge case and does not need slicing.
File CF_DATA/ERA5_01_04_1990.nc is not an edge case and does not need slicing.
File CF_DATA/ERA5_01_04_2009.nc is not an edge case and does not need slicing.
File CF_DATA/ERA5_01_04_2017.nc is not an edge case and does not need slicing.
File CF_DATA/ERA5_01_05_1986.nc is not an edge case and does not need slicing.
File

In [16]:
##Import the Test Case##

#ds = xr.open_dataset(files_sorted[8]).metpy.parse_cf()
ds = xr.open_dataset('Data_Remake_Sliced/ERA5_02_02_1965.nc').metpy.parse_cf()
#ds_sfc = ds.isel(time=slice(24, 96))  #only for edge cases
ds_sfc = ds.copy()
ds_sfc

#ds["time"].values[96:168]   #edge years day 1:   ds_sfcs ds.isel(time=slice(96, 168))
#ds["time"].values[120:192]  #edge years day 2:   ds.isel(time=slice(120, 192))
#ds["time"].values[24:96]    #edge months day 1:  ds.isel(time=slice(24, 96))
#ds["time"].values[48:120]   #edge months day 2:  ds.isel(time=slice(48, 120))

<xarray.Dataset>
Dimensions:    (longitude: 53, latitude: 41, time: 72)
Coordinates:
  * longitude  (longitude) float32 -112.0 -111.8 -111.5 ... -99.5 -99.25 -99.0
  * latitude   (latitude) float32 44.0 43.75 43.5 43.25 ... 34.5 34.25 34.0
  * time       (time) datetime64[ns] 1965-01-31 ... 1965-02-02T23:00:00
    metpy_crs  object Projection: latitude_longitude
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    d2m        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    msl        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-04-22 01:32:58 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...